In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as spark_sum, col, count, round as spark_round, lit

In [0]:
def create_frequency_table(df, column_name):
    """
    Crée une table de fréquence complète avec toutes les statistiques
    """
    total = df.count()
    
    # Calculer les fréquences
    freq_df = df.groupBy(column_name).count() \
        .withColumnRenamed('count', 'effectif') \
        .orderBy(col('effectif').desc())
    
    # Ajouter statistiques
    freq_df = freq_df.withColumn('total', lit(total)) \
        .withColumn('frequence_relative', spark_round(col('effectif') / col('total'), 4)) \
        .withColumn('pourcentage', spark_round((col('effectif') / col('total')) * 100, 2))
    
    # Fréquence cumulée
    window_spec = Window.orderBy(col('effectif').desc()) \
        .rowsBetween(Window.unboundedPreceding, 0)
    
    freq_df = freq_df \
        .withColumn('effectif_cumule', spark_sum('effectif').over(window_spec)) \
        .withColumn('pourcentage_cumule', 
                    spark_round((col('effectif_cumule') / col('total')) * 100, 2))
    
    # Réorganiser colonnes
    freq_df = freq_df.select(
        column_name,
        'effectif',
        'frequence_relative',
        'pourcentage',
        'effectif_cumule',
        'pourcentage_cumule'
    )
    
    return freq_df